IMPORTANT: for this code use data0 and data1 without separating testing data into a separate folder

<div style="width: 1350px; height: 30px; background-color: green;"></div>


# 0. Load Modules

Convert to py:

`jupyter nbconvert --to script RCNN_crossval.ipynb`

In [ ]:
# Core Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Operating System Interaction
import os
import sys

# Machine Learning Frameworks
import torch
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader

# Data Transformation and Augmentation (not all of these transformations were finally used)
from torchvision.transforms import Compose, RandomHorizontalFlip, RandomRotation, \
    RandomVerticalFlip, ColorJitter, RandomAffine, RandomPerspective, RandomResizedCrop, \
    GaussianBlur, RandomAutocontrast
from torchvision.transforms import functional as F

# Model Building and Initialization
import torch.nn as nn
from torch.nn.init import kaiming_normal_

# Data Loading and Dataset Handling
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, Subset
from PIL import Image

# Cross-Validation and Metrics
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, roc_curve, auc, accuracy_score
from scipy.special import expit as sigmoid

# Visualization and Display
from matplotlib.animation import FuncAnimation
from matplotlib.colors import Normalize
from IPython.display import HTML

# Miscellaneous
import random
from tqdm import tqdm

In [ ]:
# For Google Colab, mount Google Drive, for local environments, get local path (github)

# Change with the appropriate path. Log in into Drive and create the folders with the data

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    # Carlos
    #folder0_path = '/content/drive/My Drive/solar_jets/data0'
    #folder0_test_path = '/content/drive/My Drive/solar_jets/data0_test'
    #folder1_path = '/content/drive/My Drive/solar_jets/data1'
    #folder1_test_path = '/content/drive/My Drive/solar_jets/data1_test'

    # Julie
    folder0_path = '/content/drive/My Drive/Colab Notebooks/CS433-project2/data0'
    folder1_path = '/content/drive/My Drive/Colab Notebooks/CS433-project2/data1'
    folder0_test_path = '/content/drive/My Drive/Colab Notebooks/CS433-project2/data0_test'
    folder1_test_path = '/content/drive/My Drive/Colab Notebooks/CS433-project2/data1_test'
    folder0_valid_path = '/content/drive/My Drive/Colab Notebooks/CS433-project2/data1_val'
    folder1_valid_path = '/content/drive/My Drive/Colab Notebooks/CS433-project2/data1_val'

    #Adrien
    #folder0_path = '/content/drive/My Drive/Machine Learning/Projet/data0'
    #folder1_path = '/content/drive/My Drive/Machine Learning/Projet/data1'
    #folder0_test_path = '/content/drive/My Drive/Machine Learning/Projet/data0_test'
    #folder1_test_path = '/content/drive/My Drive/Machine Learning/Projet/data1_test'
    #folder0_valid_path = '/content/drive/My Drive/Machine Learning/Projet/data0_val'
    #folder1_valid_path = '/content/drive/My Drive/Machine Learning/Projet/data1_val'
else:
    # For local environments like VS Code
    folder0_path = '../data/data separated/data0'
    folder1_path = '../data/data separated/data1'
    folder0_test_path = '../data/data separated/data0_test'
    folder1_test_path = '../data/data separated/data1_test'
    folder0_valid_path = '../data/data separated/data0_val'
    folder1_valid_path = '../data/data separated/data1_val'

## 1.1. Prepare the dataset

#### Create the class

In [ ]:
class TensorTransforms:
    def __init__(self, rotate_angle=30):
        self.rotate_angle = rotate_angle

    def random_horizontal_flip(self, x):
        if random.random() > 0.5:
            return torch.flip(x, [2])  # Flip along width
        return x

    def random_vertical_flip(self, x):
        if random.random() > 0.5:
            return torch.flip(x, [1])  # Flip along height
        return x

    def random_rotation(self, x):
        # Random rotation in increments of 90 degrees for simplicity
        k = random.randint(0, 3)  # 0, 90, 180, or 270 degrees
        return torch.rot90(x, k, [1, 2])  # Rotate along height and width

    def __call__(self, x):
        x = self.random_horizontal_flip(x)
        x = self.random_vertical_flip(x)
        x = self.random_rotation(x)
        return x

class NPZDataset(Dataset):
    def __init__(self, data_dir, augment=True, mean=None, std=None):
        self.data_dir = data_dir
        self.augment = augment
        self.files = [f for f in os.listdir(data_dir) if self._check_file_shape(f)]
        self.transform = TensorTransforms()
        self.mean = mean
        self.std = std

    def _check_file_shape(self, file):
        file_path = os.path.join(self.data_dir, file)
        data = np.load(file_path)['arr_0']
        return data.shape == (166, 166, 30)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.files[idx])
        data = np.load(file_path)['arr_0']
        data = np.moveaxis(data, -1, 0)  # Move channel to first dimension

        data = torch.from_numpy(data).float()  # Convert to PyTorch tensor

        if self.augment:
            data = self.transform(data)

        if self.mean is not None and self.std is not None:
            data = np.clip(data, a_min=None, a_max= 1000)
            data = (data - self.mean) / self.std


        label = 1.0 if 'data1' in self.data_dir else 0.0

        return data, np.float32(label)

#### Run to get the Data

We calculated the mean and std previously with function compute_mean_std

In [ ]:
mean_data = 50.564544677734375
std_data = 49.94772720336914

In [ ]:
train_data1 = NPZDataset(folder1_path, mean=mean_data, std=std_data, augment=True)
train_data0 = NPZDataset(folder0_path, mean=mean_data, std=std_data, augment=True)
train_data = torch.utils.data.ConcatDataset([train_data1, train_data0])

In [ ]:
# Just for debugging
# train_data1 = ([train_data1[i] for i in range(100)])
# train_data0 = ([train_data0[i] for i in range(100)])
# train_data = torch.utils.data.ConcatDataset([train_data1, train_data0])

Below we'll see how train_data can be treated just like lists

- One element in the list for each sequence of images
- Each element of the list has a tuple of two elements, the first the array (166,30,30) and the second the label

In [ ]:
print("samples y=1: ",len(train_data1))
print("samples y=0: ",len(train_data0))
print("samples total ",len(train_data))

Comput mean and std for train dataset (do not run this)

In [ ]:
def compute_mean_std(dataset):
    mean = 0.0
    std = 0.0
    for data, _ in dataset:
        mean += data.mean()
        std += data.std()
    mean /= len(dataset)
    std /= len(dataset)
    return mean.item(), std.item()

# mean, std = compute_mean_std(train_data)
# mean, std

#### Divide train, validate, and test data

In [ ]:
train_size = int(0.7 * len(train_data))  # 70% of data for training
validate_size = int(0.15 * len(train_data))  # 15% for validation
test_size = len(train_data) - train_size - validate_size  # remaining 15% for testing

train_dataset, validate_dataset, test_dataset = random_split(train_data, [train_size, validate_size, test_size])

In [ ]:
print("train: ",len(train_dataset))
print("validate: ",len(validate_dataset))
print("test: ",len(test_dataset))

In [ ]:
index = 12
your_array = train_data[index][0].numpy()
print(train_data[index][1])

# Function to update the plot
def update(frame):
    im.set_array(your_array[frame])
    return [im]

# Set up the plot
fig, ax = plt.subplots()
norm = Normalize(vmin=your_array.min(), vmax=your_array.max())  # Adjust as needed
im = ax.imshow(your_array[0], cmap='hot', norm=norm)  # Initialize with the first frame

# Create the animation
ani = FuncAnimation(fig, update, frames=range(30), blit=True)

# Display as HTML5 video in the notebook
HTML(ani.to_jshtml())

DataLoader is designed to iterate over batches of data rather than individual samples, so when we try to access for example the first mini-batch as `train_loader[0]`, we get an error.

However, note how before we could iterate over it


# 2. Define the Neural Network

In [ ]:
class RCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.max_pool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.layer1 = nn.Sequential(
            nn.Conv2d(30,64,kernel_size=3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64,64,kernel_size=3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size=3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(128,128,kernel_size=3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(128,256,kernel_size=3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        self.layer6 = nn.Sequential(
            nn.Conv2d(256,256,kernel_size=3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        self.layer7 = nn.Sequential(
            nn.Conv2d(256,512,kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True)
        )
        self.layer8 = nn.Sequential(
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True)
        )
        # hidden_size is an hyperparameter to be adjusted
        # try augmenting num_layers
        self.lstm = nn.LSTM(input_size=12800, hidden_size=256, num_layers=1, batch_first=True)

        self.classifier = nn.Sequential(
            nn.Linear(256,128),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(128,32),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(32,1),
            #nn.Sigmoid()     # don't include it as it is already included in BCELogitLoss (BCELoss is less stable)
        )

        # for m in self.modules():
        #     if not RCNN:
        #         kaiming_normal_(m.weight,nonlinearity="relu")#Kaiming to initialize the weights
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.max_pool(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.max_pool(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.max_pool(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.max_pool(out)
        out = self.layer8(out)
        out = self.layer8(out)
        out = self.max_pool(out)
        out = out.view(out.size(0),-1)
        lstm_out, _ = self.lstm(out)
        out = self.classifier(lstm_out)

        return out

    def graph(self): #for visualization and debugging
        return nn.Sequential(self.layer1,self.layer2,self.maxPool,self.layer3,self.layer4,self.maxPool,self.layer5,self.layer6,self.maxPool,self.layer7,self.layer8, self.maxPool,self.layer8,self.layer8,self.maxPool,self.classifier)

# 3. Training Parameters

In [ ]:
def plot_lr(epochs, lrs):
    # Creating subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

    # Linear scale plot
    ax1.plot(epochs, lrs, marker='o')
    ax1.set_title('Learning Rate per Epoch (Linear Scale)')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Learning Rate')
    ax1.set_xticks(epochs)
    ax1.grid(True)

    # Log scale plot
    ax2.plot(epochs, lrs, marker='o')
    ax2.set_title('Learning Rate per Epoch (Log Scale)')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Learning Rate')
    ax2.set_yscale('log')
    ax2.set_xticks(epochs)
    ax2.grid(True)

    # Display the plots
    plt.show()

In [ ]:
num_epochs = 100
batch_size = 2

initial_lr = 1e-4
weight_decay = 5e-3

threshold = 0.5

model = RCNN()
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Function to create optimizer and scheduler
def reset_optimizer_scheduler():
    optimizer = torch.optim.AdamW(model.parameters(), lr=initial_lr, weight_decay=weight_decay)
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-10, last_epoch=-1)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=3, factor=0.75)

    return optimizer, scheduler

# Initialize optimizer and scheduler
optimizer, scheduler = reset_optimizer_scheduler()
criterion = torch.nn.BCEWithLogitsLoss()

# Lists to store epoch numbers and corresponding learning rates
epochs = []
lrs = []

# Loop to update and record learning rate
for epoch in range(num_epochs):
    optimizer.step()
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    epochs.append(epoch + 1)
    lrs.append(current_lr)

plot_lr(epochs, lrs)

# 4. Train Epoch Script

In [ ]:
def train_epoch(model, optimizer, scheduler, criterion, train_loader, device, threshold):
    model.train()
    correct, train_loss, total_length = 0, 0, 0

    #for i, (data, target) in tqdm(enumerate(train_loader),desc="Epoch no."+str(epoch)):
    for i, (data, target) in enumerate(train_loader):

        #MOVING THE TENSORS TO THE CONFIGURED DEVICE
        data, target = data.to(device), target.to(device).unsqueeze(1).float()

        #FORWARD PASS
        output = model(data)
        loss = criterion(output, target)

        #BACKWARD AND OPTIMIZE
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # PREDICTIONS
        threshold = 0.5
        pred = (torch.sigmoid(output) >= threshold).float()

        # PERFORMANCE CALCULATION
        train_loss += loss.item() * len(data)
        total_length += len(data)
        correct += pred.eq(target.view_as(pred)).sum().item()

    scheduler.step()
    train_loss = train_loss / total_length
    train_acc = correct / total_length

    return train_loss, train_acc, scheduler.get_last_lr()[0]

# 5. Test Script

In [ ]:
def predict(model, train_loader, criterion, device, threshold):
    model.eval()
    correct, val_loss, total_length = 0, 0, 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in train_loader:

            #MOVING THE TENSORS TO THE CONFIGURED DEVICE
            data, target = data.to(device), target.to(device).unsqueeze(1).float()

            #FORWARD PASS
            output = model(data)
            loss = criterion(output, target)
            # PREDICTIONS
            threshold = 0.5
            pred = (torch.sigmoid(output) >= threshold).float()

            # PERFORMANCE CALCULATION
            val_loss += loss.item() * len(data)
            total_length += len(data)
            correct += pred.eq(target.view_as(pred)).sum().item()
            all_preds.extend(pred.view(-1).cpu().numpy())
            all_targets.extend(target.view(-1).cpu().numpy())

    val_loss = val_loss / total_length
    val_acc = correct / total_length

    return val_loss, val_acc, np.array(all_preds), np.array(all_targets)


def test(model, train_loader, criterion, device, threshold):
    model.eval()
    correct, val_loss, total_length = 0, 0, 0
    all_preds = []
    all_targets = []
    all_out = []

    with torch.no_grad():
        for data, target in train_loader:

            #MOVING THE TENSORS TO THE CONFIGURED DEVICE
            data, target = data.to(device), target.to(device).unsqueeze(1).float()

            #FORWARD PASS
            output = model(data)
            loss = criterion(output, target)
            # PREDICTIONS
            threshold = 0.5
            pred = (torch.sigmoid(output) >= threshold).float()

            # PERFORMANCE CALCULATION
            val_loss += loss.item() * len(data)
            total_length += len(data)
            correct += pred.eq(target.view_as(pred)).sum().item()
            all_preds.extend(pred.view(-1).cpu().numpy())
            all_targets.extend(target.view(-1).cpu().numpy())
            all_out.extend(output.view(-1).cpu().numpy())

    val_loss = val_loss / total_length
    val_acc = correct / total_length

    return val_loss, val_acc, np.array(all_preds), np.array(all_targets), np.array(all_out)

# 6. Train the Network 😀😀😀😀😀


In [ ]:
# Model
model = RCNN() #creates an instance of the RCNN

# Loading in consequence

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(validate_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Hyperparameters

optimizer, scheduler = reset_optimizer_scheduler()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Logs results
loss_history_train=[]
lr_history_train=[]
acc_history_train=[]

acc_history_test=[]
loss_history_test=[]

f1_val_history = []


# Early stopping parameters
patience = 101
best_loss = float('inf')
epochs_no_improve = 0

# Save best f1 model
best_model_state = None
best_f1_score = 0


# RUN MODEL

for epoch in range(1,num_epochs+1):

  # Train
  train_loss, train_acc, lrs = train_epoch(model, optimizer, scheduler, criterion, train_loader, device, threshold)
  loss_history_train.append(train_loss)
  acc_history_train.append(train_acc)
  lr_history_train.append(lrs)

  # Validate
  val_loss, val_acc, val_preds, val_targets = predict(model, valid_loader, criterion, device, threshold)
  loss_history_test.append(val_loss)
  acc_history_test.append(val_acc)

  # Calculate F1 score
  f1 = f1_score(val_targets, val_preds, average='binary')  # adjust the average parameter as needed
  f1_val_history.append(f1)

  print(f"Epoch {epoch}/{num_epochs} - LR: {lrs:.3e}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Val F1 Score: {f1:.4f}")

  # Early stopping
  if val_loss < best_loss:
    best_loss = val_loss
    epochs_no_improve = 0
  else:
    epochs_no_improve += 1

  if epochs_no_improve == patience:
    print(f"Early stopping triggered after {epoch + 1} epochs.")
    break

  # Save Model with best F1
  if f1 == max(f1_val_history):
      best_f1_score = f1
      best_model_state = model.state_dict()  # Save the model state

In [ ]:
# Evaluate on TEST data
#test_loader = DataLoader(test_dataset, batch_size=batch_size, sampler=train_subsampler)

model.load_state_dict(best_model_state)
model.to(device)

test_loss, test_accuracy, test_predictions, test_targets, test_out = test(model, test_loader, criterion, device, threshold)
test_f1_score = f1_score(test_targets, test_predictions, average='binary')  # adjust as needed

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1_score:.4f}")

In [ ]:
accuracy_history=acc_history_test.copy()
loss_history=loss_history_test
train_loss = loss_history_train.copy()

# Use numpy's convolve function to calculate the moving average
moving_avg_train = np.convolve(train_loss, np.ones(100)/100, mode='valid')
#print(len(train_loss))
#print(len(moving_avg_train))

n_train = len(accuracy_history)

t_test = np.arange(1, num_epochs + 1)
t_train = np.linspace(1,num_epochs,len(loss_history_train))

fig, axs = plt.subplots(1, 3, figsize=(20, 5))

# Plotting training loss history
axs[0].semilogy(t_train, train_loss, color='orange', label="training loss")
#axs[0].semilogy((t_train[:-99]), moving_avg_train, color='blue', label="100-values moving avg")
axs[0].legend()
axs[0].set_title('Train loss History')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')

axs[1].plot(t_test, accuracy_history, label="Validation", color='orange')
axs[1].set_title('Validation Accuracy History')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy')
axs[1].legend()

axs[2].plot(t_test, lr_history_train)
axs[2].set_title('Learning Rate History')
axs[2].set_xlabel('Epoch')
axs[2].set_ylabel('Learning Rate')

plt.tight_layout()
plt.show()


In [ ]:
# Compute ROC curve and ROC area for each class
fpr, tpr, _ = roc_curve(test_targets, test_out)
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


# Area Under the Curve (AUC)
print("Area Under the Curve (AUC): ", roc_auc)

In [ ]:
# F1 vs threshold
thresholds = np.linspace(0.03, 0.97, 100)  # 100 thresholds between 0 and 1
f1_scores = []
accuracy_scores = []

probabilities = sigmoid(test_out)

for thresh in thresholds:
    # Convert probabilities to binary predictions based on the threshold
    predictions = (probabilities > thresh).astype(int)

    # Compute F1 score and Accuracy for this threshold
    f1_scores.append(f1_score(test_targets, predictions))
    accuracy_scores.append(accuracy_score(test_targets, predictions))



plt.figure(figsize=(12, 5))

# F1 Score subplot
plt.subplot(1, 2, 1)
plt.plot(thresholds, f1_scores, marker='')
plt.title('F1 Score vs. Thresholds')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.grid()

# Accuracy Score subplot
plt.subplot(1, 2, 2)
plt.plot(thresholds, accuracy_scores, marker='', color='green')
plt.title('Accuracy vs. Thresholds')
plt.xlabel('Threshold')
plt.ylabel('Accuracy')
plt.grid()

plt.tight_layout()
plt.show()

# As data is balanced, f1 and accuracy are similar